# Setup Logging & Imports

In [ ]:
DEBUG = True

### Setting up logging

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("pipeline")

# Show info messages if DEBUG mode is enabled
if DEBUG:
    logger.setLevel(logging.DEBUG)
    logger.debug("DEBUG mode is enabled. Detailed logs will be shown.")
else:
    logger.setLevel(logging.INFO)
    logger.info("DEBUG mode is disabled. Only essential logs will be shown.")

### Imports

In [ ]:
import os
import torch

# We set up CUDA first to ensure it is configured correctly
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
if torch.cuda.is_available():
    CUDA_DEVICE = torch.device("cuda:0")
    logger.info(f"CUDA is available. Using device: {CUDA_DEVICE}")
else:
    logger.error("CUDA is not available. Please check your PyTorch installation. Using CPU instead...this will be slow.")
    CUDA_DEVICE = torch.device("cpu")

In [ ]:
from pipeline.proj import load_projection_mat, reformat_sinogram, interpolate_projections, pad_and_reshape, divide_sinogram
from pipeline.aggregate_prj import aggregate_saved_projections
from pipeline.aggregate_ct import aggregate_saved_recons
from pipeline.apply_model import apply_model_to_projections, load_model, apply_model_to_recons
from pipeline.utils import ensure_dir, read_scans_agg_file
from pipeline.paths import Directories, Files
import scipy.io
import matlab.engine
import matplotlib.pyplot as plt
import numpy as np
import yaml
import importlib
import copy
from tqdm import tqdm
import gc
import tigre.utilities.gpu as gpu
from pipeline.FDK_half.FDK_half import FDKHalf
from pipeline.utils import get_geometry

# TODO run FDK via: FFrecon_reconFDK(input_mat, output_mat); in file "FFrecon_fullFDK.m"

# Configuration

In [ ]:
# Scans to convert to PyTorch tensors
# Put None if you don't have any scans to convert
# See the README for how to write this file correctly
# NOTE: This will throw an error if the scan has already been converted
#       If you would like to re-convert a scan,
#       you can delete the file manually
# SCANS_CONVERT = 'scans_convert_to_pt.txt'
SCANS_CONVERT = None

# Phase of the project (all data, models, etc. will be saved under this phase)
PHASE = "7"

# If this data version already exists in this phase, it will be loaded
# Otherwise it will be created using whatever the most updated data creation script is
DATA_VERSION = '13'

# Scans to use for training, val, and testing
# You should set this even if you are not doing aggregation
# See the README for how to write this file correctly
# NOTE: This will NOT throw an error if there are already aggregated scans
#       it will just give a warning and skip the aggregation step
SCANS_AGG = 'scans_to_agg.txt'
# SCANS_AGG = None

# Whether to augment the data for the image domain
# This will only be used if you are doing image domain aggregation
AUGMENT_ID = True

# List of yaml files that contain configurations for the pipeline
# Each file should contain the paramters for a specific model/ensemble
CONFIG_FILES = [
    "config.yaml",
    # "config2.yaml",
]

# Base directory
WORK_ROOT = "E:/NoahSilverberg/ngCBCT"

# NSG_CBCT Path where the raw matlab data is stored
NSG_CBCT_PATH = "D:/MitchellYu/NSG_CBCT"

# Directory with all files specific to this phase/data version
PHASE_DATAVER_DIR = os.path.join(
    WORK_ROOT, f"phase{PHASE}", f"DS{DATA_VERSION}"
)

DIRECTORIES = Directories(
    # mat_projections_dir=os.path.join(WORK_ROOT, "mat"),
    # pt_projections_dir=os.path.join("I:/", "prj_pt"),
    # projections_aggregate_dir=os.path.join(PHASE_DATAVER_DIR, "aggregates", "projections"),
    # projections_model_dir=os.path.join(PHASE_DATAVER_DIR, "models", "projections"),
    # projections_results_dir=os.path.join(PHASE_DATAVER_DIR, "results", "projections"),
    # projections_gated_dir=os.path.join(WORK_ROOT, "gated", "prj_mat"),
    reconstructions_dir=os.path.join(PHASE_DATAVER_DIR, "reconstructions"),
    reconstructions_gated_dir=os.path.join(WORK_ROOT, "gated", "fdk_recon"),
    images_aggregate_dir=os.path.join(PHASE_DATAVER_DIR, "aggregates", "images"),
    images_model_dir=os.path.join('H:\Public/Noah/phase7/DS13', "models", "images"),
    images_results_dir=os.path.join('H:\Public/Noah/phase7/DS13', "results", "images"),
)

FILES = Files(DIRECTORIES)

# Data Preparation: projection interpolation

In [ ]:
if SCANS_CONVERT is not None:
    # Read the scans to convert file
    with open(SCANS_CONVERT, "r") as f:
        SCANS_CONVERT = []
        for line in f:
            line = line.strip()
            if not line:
                continue
            patient, scan, scan_type = line.split()
            SCANS_CONVERT.append((patient, scan, scan_type))

    logger.debug(f"Loaded scan list for conversion: {SCANS_CONVERT}")

    logger.info("Starting to process projection data...")

    for patient, scan, scan_type in SCANS_CONVERT:
        g_path = FILES.get_projection_pt_filepath(patient, scan, scan_type, gated=True)
        ng_path = FILES.get_projection_pt_filepath(patient, scan, scan_type, gated=False)

        # Make sure the files do not already exist
        if os.path.exists(g_path) or os.path.exists(ng_path):
            logger.warning(f"Projection files already exist for patient {patient}, scan {scan}, type {scan_type}. Skipping...")
            continue

        # Load the projection data from the matlab files
        mat_path = FILES.get_projection_mat_filepath(patient, scan, scan_type)
        odd_index, angles, prj = load_projection_mat(mat_path)

        # Log shapes of loaded data
        logger.debug(f'Processing patient {patient}, scan {scan}, type {scan_type}')
        logger.debug(f'Loaded odd_index shape: {odd_index.shape}')
        logger.debug(f'Loaded angles shape: {angles.shape}')
        logger.debug(f'Loaded projection shape: {prj.shape}')

        # Flip and permute to get it in the right format
        prj_gcbct, angles1 = reformat_sinogram(prj, angles)

        # Log shapes after reformatting
        logger.debug(f'Reformatted projection shape: {prj_gcbct.shape}')

        # Simulate ngCBCT projections
        prj_ngcbct_li = interpolate_projections(prj_gcbct, odd_index)

        # Log shapes after interpolation
        logger.debug(f'Interpolated ngCBCT projection shape: {prj_ngcbct_li.shape}')

        # Split the projections into two halves so they are good dimensions for the CNN
        combined_gcbct = divide_sinogram(pad_and_reshape(prj_gcbct), v_dim=512 if scan_type == "HF" else 256)
        combined_ngcbct = divide_sinogram(pad_and_reshape(prj_ngcbct_li), v_dim=512 if scan_type == "HF" else 256)

        # Log shapes after dividing sinograms
        logger.debug(f'Combined gCBCT shape: {combined_gcbct.shape}')
        logger.debug(f'Combined ngCBCT shape: {combined_ngcbct.shape}')

        logger.debug(f'Saving projections...')
        
        # Save the projections
        torch.save(combined_gcbct, g_path)
        torch.save(combined_ngcbct, ng_path)

        logger.debug(f'Done with patient {patient}, scan {scan}, type {scan_type}\n')

    logger.info("All projections saved successfully.")

    # Free up memory
    del odd_index, angles, prj, prj_gcbct, angles1, prj_ngcbct_li, combined_gcbct, combined_ngcbct
else:
    logger.info("No scans to convert. Skipping projection data processing.")

### DEBUG: Sample projections

In [ ]:
if DEBUG and SCANS_CONVERT is not None:
    # Pick the first HF scan and first FF scan
    hf_scan = None
    ff_scan = None
    for patient, scan, scan_type in SCANS_CONVERT:
        if scan_type == "HF":
            hf_scan = (patient, scan, scan_type)
            break
    for patient, scan, scan_type in SCANS_CONVERT:
        if scan_type == "FF":
            ff_scan = (patient, scan, scan_type)
            break

    # Display the first HF scan
    # Show the gated and nonstop-gated on subplots
    if hf_scan:
        hf_patient, hf_scan_num, hf_scan_type = hf_scan
        g_path = FILES.get_projection_pt_filepath(hf_patient, hf_scan_num, hf_scan_type, gated=True)
        ng_path = FILES.get_projection_pt_filepath(hf_patient, hf_scan_num, hf_scan_type, gated=False)
        hf_gated_prj = torch.load(g_path)
        hf_ng_prj = torch.load(ng_path)
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(hf_gated_prj[0, 0, :, :].cpu().numpy(), cmap='gray')
        plt.title(f'Gated Projection - {hf_scan_type} p{hf_patient}_{hf_scan_num}')
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(hf_ng_prj[0, 0, :, :].cpu().numpy(), cmap='gray')
        plt.title(f'Nonstop-Gated Projection - {hf_scan_type} p{hf_patient}_{hf_scan_num}')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

        # Free up memory
        del hf_gated_prj, hf_ng_prj

    # Repeat for FF scan
    if ff_scan:
        ff_patient, ff_scan_num, ff_scan_type = ff_scan
        g_path = FILES.get_projection_pt_filepath(ff_patient, ff_scan_num, ff_scan_type, gated=True)
        ff_gated_prj = torch.load(g_path)
        ng_path = FILES.get_projection_pt_filepath(ff_patient, ff_scan_num, ff_scan_type, gated=False)
        ff_ng_prj = torch.load(ng_path)
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(ff_gated_prj[0, 0, :, :].cpu().numpy(), cmap='gray')
        plt.title(f'Gated Projection - {ff_scan_type} p{ff_patient}_{ff_scan_num}')
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(ff_ng_prj[0, 0, :, :].cpu().numpy(), cmap='gray')
        plt.title(f'Nonstop-Gated Projection - {ff_scan_type} p{ff_patient}_{ff_scan_num}')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

        # Free up memory
        del ff_gated_prj, ff_ng_prj

# Aggregate projections for train/val/test

In [ ]:
if SCANS_AGG is not None:
    scans_agg, scan_type = read_scans_agg_file(SCANS_AGG)
    logger.debug(f"Loaded scan list for aggregation: {scans_agg}")

    # Only aggregate projections if they don't already exist
    agg_dir = DIRECTORIES.projections_aggregate_dir
    if agg_dir is None:
        logger.warning("No aggregation directory specified. Skipping projection data aggregation.")
    elif len(os.listdir(agg_dir)) > 0:
        logger.warning(f"Aggregated projection data already exists in {agg_dir}. Skipping...")
    else:
        logger.info("Starting to aggregate projection data...")
        
        # Aggregate and save projection data sets
        for split in ['TRAIN', 'VALIDATION', 'TEST']:
            if len(scans_agg[split]) > 0:
                ng_paths = [FILES.get_projection_pt_filepath(patient, scan, scan_type, gated=False) for patient, scan, scan_type in scans_agg[split]]
                prj_ngcbct = aggregate_saved_projections(ng_paths)
                ng_agg_path = FILES.get_projections_aggregate_filepath(split, gated=False)
                np.save(ng_agg_path, prj_ngcbct.numpy())
                del prj_ngcbct
                logger.debug("Done with nonstop-gated...")
                g_paths = [FILES.get_projection_pt_filepath(patient, scan, scan_type, gated=True) for patient, scan, scan_type in scans_agg[split]]
                prj_gcbct = aggregate_saved_projections(g_paths)
                g_agg_path = FILES.get_projections_aggregate_filepath(split, gated=True)
                np.save(g_agg_path, prj_gcbct.numpy())
                del prj_gcbct
                logger.debug("Done with gated...")

                logger.debug(f"Aggregated projections saved for {scan_type} {split}.\n")
            else:
                logger.debug(f"No scans to aggregate for {scan_type} {split}. Skipping aggregation.")

        # Free up memory
        del prj_gcbct, prj_ngcbct

        logger.info("Projection data aggregation completed successfully.")
        logger.info("Aggregated projection data saved in: %s", agg_dir)
else:
    logger.info("No scans to aggregate. Skipping projection data aggregation.")

# Training PD CNN

In [ ]:
for config_file in CONFIG_FILES:
    # Load the yaml configuration file
    with open(config_file, "r") as f:
        config = yaml.safe_load(f)
    
    logger.debug(f"Loaded configuration from {config_file}")

    # Skip this config if the user has set PD_training to False
    if not config['PD_settings']['training']:
        logger.info(f"Skipping PD training for {config_file} as PD training is set to False.")
        continue

    # Get the training application
    module_name, class_name = config['PD_settings']['training_app'].rsplit('.', 1)
    module = importlib.import_module("pipeline." + module_name)
    cls = getattr(module, class_name)

    logger.debug(f"Loaded class {class_name} from module {module_name}")

    # Get the model version (for naming purposes)
    model_version = config['PD_settings']['model_version']

    # Get the ensemble size, and loop through it
    ensemble_size = config['PD_settings']['ensemble_size']
    for i in range(ensemble_size):
        # If we are training an ensemble, we add an identifier to the model version
        if ensemble_size > 1:
            # Deepcopy config so we don't affect the original
            cfg = copy.deepcopy(config)
            cfg['PD_settings']['model_version'] = f"{model_version}_{i+1:02}" # e.g., "v1_01"
        else:
            cfg = config

        # Add the data version to the configuration
        cfg['PD_settings']['data_version'] = DATA_VERSION

        # checkpoint = FILES.get_model_filepath(model_version=cfg['PD_settings']['model_version'], domain='PROJ', checkpoint=1, ensure_exists=False)
        # if os.path.exists(checkpoint):
        #     checkpoint = torch.load(checkpoint)
        #     epoch = checkpoint['epoch']
        #     state_dict = checkpoint['state_dict']
        #     optimizer = checkpoint['optimizer']

        #     logger.info(f"Resuming training from epoch {epoch} for model version {cfg['PD_settings']['model_version']}...")

        #     # Instantiate with the loaded configuration
        #     instance = cls(cfg, "PROJ", DEBUG, FILES, epoch, optimizer, state_dict)
        # else:
        #     logger.info(f"Starting training from scratch for model version {cfg['PD_settings']['model_version']}...")
        #     instance = cls(cfg, "PROJ", DEBUG, FILES)

        # Instantiate with the loaded configuration
        instance = cls(cfg, "PROJ", DEBUG, FILES)

        logger.info(f"Going to try training the {i + 1}-th model with configuration from {config_file}...")

        # Run the training
        instance.main()

        logger.info(f"Finished training the {i + 1}-th model.\n")

        del instance, cfg
        gc.collect()

    # Free up memory
    del module, cls, config, module_name, class_name

# Apply PD model and FDK to all scans

In [ ]:
if SCANS_AGG is None:
    logger.info("Skipping model application as the aggregation scan list is not provided.")
else:
    eng = matlab.engine.start_matlab()

    matlab_script_path = 'D:/NoahSilverberg/CudaRecon'
    cuda_tools = r'D:\NoahSilverberg\CudaTools'
    matlab_functions = r'D:\NoahSilverberg\CommonMatlabFunctions_HZ'

    eng.addpath(cuda_tools, nargout=0)
    eng.addpath(matlab_functions, nargout=0)
    eng.addpath(matlab_script_path, nargout=0)

    # Loop through the configurations again
    for config_file in CONFIG_FILES:
        # Load the yaml configuration file
        with open(config_file, "r") as f:
            config = yaml.safe_load(f)

        logger.debug(f"Loaded configuration from {config_file}")

        # Get the ensemble size, and loop through it
        ensemble_size = config['PD_settings']['ensemble_size']
        for i in range(ensemble_size):
            model_version = config['PD_settings']['model_version']

            # If we are training an ensemble, we add an identifier to the model version
            if ensemble_size > 1:
                model_version = f"{model_version}_{i+1:02}"

            scan_type_model = config['PD_settings']['scan_type']

            # Load the trained PD model onto the GPU
            model_path = FILES.get_model_filepath(model_version, "PROJ")
            PD_model = load_model(config['PD_settings']['network_name'], config['PD_settings']['network_kwargs'], model_path, CUDA_DEVICE)

            passthrough_count = config['PD_settings']['passthrough_count']

            scans_agg, scan_type = read_scans_agg_file(SCANS_AGG)
            if scan_type != config['PD_settings']['scan_type']:
                raise ValueError(f"Scan type in aggregation file ({scan_type}) does not match scan type in config ({config['PD_settings']['scan_type']}).")
            
            for split in ['VALIDATION', 'TEST']:
                for patient, scan, scan_type in tqdm(scans_agg[split], desc=f"Applying model {model_version} to projections split {split}"):
                    # Get the matlab dicts for the ground truth and nostop-gated projections
                    mat_path = FILES.get_projection_mat_filepath(patient, scan, scan_type)
                    gated_pt_path = FILES.get_projection_pt_filepath(patient, scan, scan_type, gated=True)
                    ng_pt_path = FILES.get_projection_pt_filepath(patient, scan, scan_type, gated=False)

                    # Get the acquired nonstop-gated indices and angles from the .mat file
                    # NOTE: excluding prj speeds it speeds up a bit
                    odd_index, angles = load_projection_mat(
                        mat_path, exclude_prj=True
                    )

                    # Load the gated and (interpolated) nonstop-gated projections
                    # NOTE: These each have shape (2*H, 1, v_dim, 512)
                    prj_gcbct = torch.load(gated_pt_path).detach()
                    prj_ngcbct_li = torch.load(ng_pt_path).detach()

                    for passthrough_num in range(passthrough_count):
                    
                        # Save paths for the the ground truth and CNN-processed projections
                        gated_path = FILES.get_projections_results_filepath(model_version, patient, scan, scan_type, gated=True)
                        ng_path = FILES.get_projections_results_filepath(model_version, patient, scan, scan_type, gated=False, passthrough_num=passthrough_num if passthrough_count > 1 else None)
                        g_recon_path = FILES.get_recon_filepath(model_version, patient, scan, scan_type, gated=True)
                        ng_recon_path = FILES.get_recon_filepath(model_version, patient, scan, scan_type, gated=False, passthrough_num=passthrough_num if passthrough_count > 1 else None)

                        # For training we only save the reconstructions
                        if split == 'TRAIN'and os.path.exists(ng_recon_path):
                            logger.info(f"Nonstop-gated projections and reconstructions already exist for {scan_type} p{patient}_{scan} for model {model_version}. Skipping...")
                            continue

                        # For validation and testing we save both projections and reconstructions
                        if split != 'TRAIN' and os.path.exists(ng_recon_path) and os.path.exists(ng_path):
                            logger.info(f"Nonstop-gated projections and reconstructions already exist for {scan_type} p{patient}_{scan} for model {model_version}. Skipping...")
                            continue

                        if os.path.exists(ng_path):
                            cnn_mat = scipy.io.loadmat(ng_path)
                            if split == 'TRAIN':
                                os.remove(ng_path)  # Remove the .mat file to save space

                            logger.debug(f"Loaded existing nonstop-gated projections for {scan_type} p{patient}_{scan} from {ng_path}.")
                        else:
                            g_mat, cnn_mat = apply_model_to_projections(PD_model, scan_type, odd_index, angles, prj_gcbct, prj_ngcbct_li, CUDA_DEVICE, train_at_inference=config['PD_settings']['train_at_inference'], _batch_size=4)
                            logger.debug(f"Applied model {model_version} to projections for {scan_type} p{patient}_{scan}.")

                            if split != 'TRAIN':
                                scipy.io.savemat(ng_path, cnn_mat)
                                logger.debug(f"Saved nonstop-gated projections for {scan_type} p{patient}_{scan} to {ng_path}.")

                        # We only need to do & save FDK recons once
                        if not os.path.exists(g_recon_path):   
                            prj_ = matlab.single(g_mat['prj'].astype(np.float32))
                            angles_ = matlab.single(g_mat['angles'].astype(np.float32))
                            odd_index_ = matlab.single(g_mat['odd_index'].astype(np.float32))
                            del g_mat
                            g_fdk_half = np.array(eng.HFrecon_nsFDK(prj_, angles_, odd_index_, nargout=1))
                            del prj_, angles_, odd_index_

                            # Convert to Pytorch tensor
                            g_fdk_half = torch.from_numpy(g_fdk_half).detach()
                            g_fdk_half = torch.permute(g_fdk_half, (2, 0, 1))
                            
                            # Save the recon results as .pt
                            torch.save(g_fdk_half, g_recon_path)
                            del g_fdk_half
                            logger.debug(f"Saved gated reconstruction for {scan_type} p{patient}_{scan} to {g_recon_path}.")
                        else:
                            logger.debug(f"Gated reconstruction already exists for {scan_type} p{patient}_{scan}.")

                        # We only need to do & save FDK recons once
                        if not os.path.exists(ng_recon_path):
                            prj_ = matlab.single(cnn_mat['prj'].astype(np.float32))
                            angles_ = matlab.single(cnn_mat['angles'].astype(np.float32))
                            odd_index_ = matlab.single(cnn_mat['odd_index'].astype(np.float32))
                            del cnn_mat
                            ng_fdk_half = np.array(eng.HFrecon_nsFDK(prj_, angles_, odd_index_, nargout=1))
                            del prj_, angles_, odd_index_

                            # Convert to Pytorch tensor
                            ng_fdk_half = torch.from_numpy(ng_fdk_half).detach()
                            ng_fdk_half = torch.permute(ng_fdk_half, (2, 0, 1))

                            # Save the recon results as .pt
                            torch.save(ng_fdk_half, ng_recon_path)
                            del ng_fdk_half
                            logger.debug(f"Saved nonstop-gated reconstruction for {scan_type} p{patient}_{scan} to {ng_recon_path}.")
                        else:
                            logger.debug(f"Nonstop-gated reconstruction already exists for {scan_type} p{patient}_{scan}.")

                        logger.debug(f"Saved projections for {scan_type} p{patient}_{scan}.")

                    del prj_ngcbct_li, odd_index, angles

            # Free up memory
            del PD_model

    eng.quit()

    logger.info("All models applied to projections.")

# Aggregate CT volumes for train/val/test

In [ ]:
if SCANS_AGG is not None:
    scans_agg, scan_type = read_scans_agg_file(SCANS_AGG)
    logger.debug(f"Loaded scan list for aggregation: {scans_agg}")

    for config_file in CONFIG_FILES:

        # Load the yaml configuration file
        with open(config_file, "r") as f:
            config = yaml.safe_load(f)
        
        logger.debug(f"Loaded configuration from {config_file}")

        # Get the ensemble size, and loop through it
        ensemble_size = config['ID_settings']['ensemble_size']
        for i in range(ensemble_size):
            # If we are training an ensemble, we add an identifier to the model version
            if ensemble_size > 1 and config['ID_settings']['input_type_match_ensemble']:
                input_type = f"{config['ID_settings']['input_type']}_{i+1:02}" # e.g., "v1_01"
            else:
                input_type = config['ID_settings']['input_type']

            # Only aggregate reconstructions if they don't already exist
            agg_dir = DIRECTORIES.get_images_aggregate_dir(input_type)
            if len(os.listdir(agg_dir)) > 0:
                logger.warning(f"Aggregated reconstruction data already exists in {agg_dir}. Skipping...")
            else:
                logger.info("Starting to aggregate reconstruction data...")

                # Aggregate and save reconstruction data sets
                for split in ['TRAIN', 'VALIDATION', 'TEST']:
                    if len(scans_agg[split]) > 0:
                        ng_paths = [FILES.get_recon_filepath(input_type, patient, scan, scan_type, gated=False) for patient, scan, scan_type in scans_agg[split]]
                        recon_ngcbct = aggregate_saved_recons(ng_paths, augment=AUGMENT_ID)
                        ng_agg_path = FILES.get_images_aggregate_filepath(input_type, split, gated=False)
                        np.save(ng_agg_path, recon_ngcbct.numpy())
                        del recon_ngcbct
                        logger.debug("Done with nonstop-gated...")

                        g_agg_path = FILES.get_images_aggregate_filepath('fdk', split, gated=True)
                        if os.path.exists(g_agg_path):
                            logger.warning(f"Gated aggregation file {g_agg_path} already exists. Skipping aggregation for gated data.")
                            continue
                        
                        g_paths = [FILES.get_recon_filepath(input_type, patient, scan, scan_type, gated=True) for patient, scan, scan_type in scans_agg[split]]
                        recon_gcbct = aggregate_saved_recons(g_paths, augment=AUGMENT_ID)
                        np.save(g_agg_path, recon_gcbct.numpy())
                        del recon_gcbct
                        logger.debug("Done with gated...")

                        logger.debug(f"Aggregated reconstructions saved for {scan_type} {split}.\n")
                    else:
                        logger.debug(f"No scans to aggregate for {scan_type} {split}. Skipping aggregation.")

            logger.info("Reconstruction data aggregation completed successfully.")
            logger.info("Aggregated reconstruction data saved in: %s", agg_dir)
else:
    logger.info("No scans to aggregate. Skipping reconstruction data aggregation.")

# Train ID CNN

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
for config_file in CONFIG_FILES:
    # Load the yaml configuration file
    with open(config_file, "r") as f:
        config = yaml.safe_load(f)
    
    logger.debug(f"Loaded configuration from {config_file}")

    # Skip this config if the user has set ID_training to False
    if not config['ID_settings']['training']:
        logger.info(f"Skipping ID training for {config_file} as ID training is set to False.")
        continue

    # Get the training application
    module_name, class_name = config['ID_settings']['training_app'].rsplit('.', 1)
    module = importlib.import_module("pipeline." + module_name)
    cls = getattr(module, class_name)

    logger.debug(f"Loaded class {class_name} from module {module_name}")

    # Get the model version (for naming purposes)
    model_version = config['ID_settings']['model_version']

    # Get the ensemble size, and loop through it
    ensemble_size = config['ID_settings']['ensemble_size']
    for i in range(ensemble_size):
        # If we are training an ensemble, we add an identifier to the model version
        if ensemble_size > 1:
            # Deepcopy config so we don't affect the original
            cfg = copy.deepcopy(config)
            cfg['ID_settings']['model_version'] = f"{model_version}_{i+1:02}" # e.g., "v1_01"
            if config['ID_settings']['input_type_match_ensemble']:
                cfg['ID_settings']['input_type'] = f"{cfg['ID_settings']['input_type']}_{i+1:02}"
        else:
            cfg = config

        # Add the data version to the configuration
        cfg['ID_settings']['data_version'] = DATA_VERSION

        # Pass the scan list and augmentation flag to the TrainingApp
        scans_agg, _ = read_scans_agg_file(SCANS_AGG)
        cfg['ID_settings']['augment_id'] = AUGMENT_ID

        checkpoint = FILES.get_model_filepath(model_version=cfg['ID_settings']['model_version'], domain='IMAG', checkpoint=65)
        checkpoint = torch.load(checkpoint)
        epoch = checkpoint['epoch']
        state_dict = checkpoint['state_dict']
        optimizer = checkpoint['optimizer']

        # # Instantiate with the loaded configuration
        instance = cls(cfg, "IMAG", DEBUG, FILES, epoch, optimizer, state_dict, scans_agg=scans_agg)
        # instance = cls(cfg, "IMAG", DEBUG, FILES, scans_agg=scans_agg)

        logger.info(f"Going to try training the {i + 1}-th model with configuration from {config_file}...")

        # Run the training
        instance.main()

        logger.info(f"Finished training the {i + 1}-th model.\n")

        del instance, cfg
        gc.collect()

    # Free up memory
    del module, cls, config, module_name, class_name

# Pass all samples through the ID model

In [ ]:
if SCANS_AGG is None:
    logger.info("Skipping model application as the aggregation scan list is not provided.")
else:
    # Loop through the configurations again
    for config_file in CONFIG_FILES:
        # Load the yaml configuration file
        with open(config_file, "r") as f:
            config = yaml.safe_load(f)

        logger.debug(f"Loaded configuration from {config_file}")

        # Get the ensemble size, and loop through it
        ensemble_size = config['ID_settings']['ensemble_size']
        for i in range(ensemble_size):
            model_version = config['ID_settings']['model_version']
            input_type = config['ID_settings']['input_type']

            # If we are training an ensemble, we add an identifier to the model version
            if ensemble_size > 1:
                model_version = f"{model_version}_{i+1:02}"
                if config['ID_settings']['input_type_match_ensemble']:
                    input_type = f"{input_type}_{i+1:02}"

            # Load the trained ID model onto the GPU
            model_path = FILES.get_model_filepath(model_version, "IMAG")
            ID_model = load_model(config['ID_settings']['network_name'], config['ID_settings']['network_kwargs'], model_path, CUDA_DEVICE)

            passthrough_count = config['ID_settings']['passthrough_count']

            scans_agg, scan_type = read_scans_agg_file(SCANS_AGG, list_=False)
            if scan_type != config['ID_settings']['scan_type']:
                raise ValueError(f"Scan type in aggregation file ({scan_type}) does not match scan type in config ({config['ID_settings']['scan_type']}).")
            
            for patient, scan, scan_type in tqdm((scans_agg["VALIDATION"]), desc=f"Applying model {model_version} to projections"):
                for passthrough_num in range(passthrough_count):
                    ng_pt_path = FILES.get_recon_filepath(input_type, patient, scan, scan_type, gated=False, passthrough_num=passthrough_num if passthrough_count > 1 else None)
                    ng_path = FILES.get_images_results_filepath(
                        model_version, patient, scan, passthrough_num=passthrough_num if passthrough_count > 1 else None
                    )

                    if os.path.exists(ng_path):
                        logger.info(f"ID CNN results already exist for {scan_type} p{patient}_{scan} for model {model_version} (passthrough {passthrough_num}). Skipping...")
                        continue

                    recon = apply_model_to_recons(
                        ID_model, ng_pt_path, CUDA_DEVICE,
                        train_at_inference=config['ID_settings']['train_at_inference'],
                        _batch_size=8
                    )

                    torch.save(recon, ng_path)
                    logger.debug(f"Saved projections for {scan_type} p{patient}_{scan} (passthrough {passthrough_num}).")
                    del recon

            # Free up memory
            del ID_model

    logger.info("All models applied to projections.")


In [ ]:
from pipeline.network_instance import SWAG, IResNetDropout
checkpoint_paths = [FILES.get_model_filepath('MK7_MCDROPOUT_30_pct_NEW_SWAG_lr5e-4', "IMAG", checkpoint=i) for i in range(51, 66)]
frozen_layers = ['conv1', 'conv1_extra', 'up2', 'up_conv2', 'conv_1x1']
frozen_params = [name for name, param in IResNetDropout().named_parameters() if name.startswith(tuple(frozen_layers))]
print(f"Frozen parameters: {frozen_params}")
ID_model = SWAG(IResNetDropout, checkpoint_paths[:], 15, frozen_params).to(CUDA_DEVICE)

# In your passthrough script, right after ID_model is created:
# In your passthrough script, after ID_model is created:
logger.info("--- DIAGNOSTIC: Comparing SWA Mean vs. SWAG Samples ---")

# --- 1. Data Preparation ---
# This section is run once to prepare the input image.
scans_agg, scan_type = read_scans_agg_file(SCANS_AGG, list_=False)
patient, scan, scan_type = scans_agg["VALIDATION"][0]
logger.info(f"Running diagnostic on patient {patient}, scan {scan}, scan type {scan_type}...")

# Load and preprocess the scan data
input_image_full = FILES.get_recon_filepath('MK7_MCDROPOUT_30_pct_NEW', patient, scan, scan_type, gated=False, passthrough_num=0)
input_image_full = torch.load(input_image_full).to(CUDA_DEVICE)

# Extract a few slices for the batch and preprocess them
# Using a batch of slices is important for some layers, even if we visualize one.
input_batch = input_image_full[100:101, :, :] 
if input_batch.dim() == 3: # Make sure it has a channel dimension
    input_batch = input_batch.unsqueeze(1) 

# Apply clipping and scaling if necessary
input_batch = 25. * torch.clip(input_batch, 0, 0.04)
logger.info(f"Prepared input batch of shape: {input_batch.shape}")


# --- 2. Get SWA Mean Prediction ---
logger.info("Generating prediction from SWA Mean weights...")
# Manually load the SWA mean weights into the base model's parameters
with torch.no_grad():
    for module, name in ID_model.params:
        mean_param = module.get_buffer(f"{name}_mean").data
        import torch.nn as nn
        module.register_parameter(name, nn.Parameter(mean_param))

ID_model.eval()
with torch.no_grad():
    swa_mean_output = ID_model(input_batch.to(CUDA_DEVICE))


# --- 3. Get SWAG Sample Predictions ---
logger.info("Generating predictions from 3 SWAG samples...")
sampled_outputs = []
for i in range(3):
    # This call overwrites the model's parameters with a new sample
    ID_model.sample(scale=0.1) 
    ID_model.eval()
    with torch.no_grad():
        sampled_output = ID_model(input_batch.to(CUDA_DEVICE))
        sampled_outputs.append(sampled_output)
    logger.info(f"Generated output for sample {i+1}")


# --- 4. Visualize and Compare ---
import matplotlib.pyplot as plt

# Choose a single slice from the batch to visualize (e.g., the first one)
slice_index_to_show = 0 
num_images = 1 + len(sampled_outputs) # 1 for SWA mean + 3 samples

fig, axes = plt.subplots(1, num_images, figsize=(20, 5))
fig.suptitle(f'Diagnostic Comparison for Slice {100 + slice_index_to_show}', fontsize=16)

# Plot the SWA Mean output
axes[0].imshow(swa_mean_output[slice_index_to_show, 0].cpu().numpy(), cmap='gray', vmin=-0.05, vmax=1.05)
axes[0].set_title('SWA Mean Output')
axes[0].axis('off')

# Plot the SWAG Sample outputs
for i, sampled_output in enumerate(sampled_outputs):
    ax = axes[i + 1]
    ax.imshow(sampled_output[slice_index_to_show, 0].cpu().numpy(), cmap='gray', vmin=-0.05, vmax=1.05)
    ax.set_title(f'SWAG Sample {i+1}')
    ax.axis('off')

plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to make room for suptitle
plt.show()

logger.info("--- END DIAGNOSTIC ---")

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm

logger.info("--- DIAGNOSTIC: Comprehensive SWA Mean vs. SWAG Sample Analysis ---")

# --- 0. Configuration ---
num_samples = 50
num_random_samples_to_show = 3
# NOTE: You can adjust the scale here to test its effect on the artifacts
sample_scale = 1.0

# --- 1. Data Preparation ---
scans_agg, scan_type = read_scans_agg_file(SCANS_AGG, list_=False)
patient, scan, scan_type = scans_agg["TEST"][3]
logger.info(f"Running diagnostic on patient {patient}, scan {scan}, scan type {scan_type}...")

input_image_full = FILES.get_recon_filepath('MK7_MCDROPOUT_30_pct_NEW', patient, scan, scan_type, gated=False, passthrough_num=1)
input_image_full = torch.load(input_image_full).to(CUDA_DEVICE)

input_batch = input_image_full[80:81, :, :] 
if input_batch.dim() == 3:
    input_batch = input_batch.unsqueeze(1) 

input_batch = 25. * torch.clip(input_batch, 0, 0.04)
logger.info(f"Prepared input batch of shape: {input_batch.shape}")


# --- 2. Get SWA Mean Prediction ---
logger.info("Generating prediction from SWA Mean weights...")
with torch.no_grad():
    for module, name in ID_model.params:
        mean_param = module.get_buffer(f"{name}_mean").data
        module.register_parameter(name, nn.Parameter(mean_param))

ID_model.eval()
with torch.no_grad():
    swa_mean_output = ID_model(input_batch.to(CUDA_DEVICE))


# --- 3. Collect SWAG Sample Predictions ---
logger.info(f"Generating {num_samples} predictions from SWAG samples...")
sampled_outputs = []
for i in tqdm(range(num_samples), desc="Generating Samples"):
    ID_model.sample(scale=sample_scale) 
    ID_model.eval()
    with torch.no_grad():
        sampled_output = ID_model(input_batch.to(CUDA_DEVICE))
        sampled_outputs.append(sampled_output)

# Stack all sampled outputs into a single tensor for efficient computation
# Shape: [num_samples, batch_size, channels, height, width]
all_samples_tensor = torch.stack(sampled_outputs, dim=0)


# --- 4. Compute Derived Images ---
logger.info("Computing mean, std, and error maps...")
# (2) The mean of the produced pictures
mean_of_samples = torch.mean(all_samples_tensor, dim=0)

# (4) The Standard Deviation (STD) map for uncertainty
std_map = torch.std(all_samples_tensor, dim=0)

# (5) The absolute error map
abs_error_map = torch.abs(mean_of_samples - swa_mean_output)

# (3) Choose 3 random individual samples to display
random_indices = random.sample(range(num_samples), k=num_random_samples_to_show)
random_samples_to_show = [sampled_outputs[i] for i in random_indices]


# --- 5. Visualize and Compare ---
logger.info("Generating comparison plot...")
slice_index_to_show = 0 
vmin, vmax = -0.05, 1.05 # Use consistent scaling for grayscale images

fig, axes = plt.subplots(2, 4, figsize=(24, 12))
fig.suptitle(f'Comprehensive SWAG Diagnostic for Slice {100 + slice_index_to_show} ({num_samples} Samples, Scale={sample_scale})', fontsize=20)

# Plotting function to reduce boilerplate
def plot_img(ax, img_data, title, cmap='gray', vlims=(vmin, vmax)):
    if vlims:
        im = ax.imshow(img_data, cmap=cmap, vmin=vlims[0], vmax=vlims[1])
    else: # For error/std maps, use automatic scaling
        im = ax.imshow(img_data, cmap=cmap)
    ax.set_title(title, fontsize=14)
    ax.axis('off')
    # add colorbar
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

# Row 1
plot_img(axes[0, 0], input_batch[slice_index_to_show, 0][:, :].cpu().numpy(), "Input Slice", vlims=None, cmap='gray')
plot_img(axes[0, 1], swa_mean_output[slice_index_to_show, 0][:, :].cpu().numpy(), "(1) SWA Mean Output")
plot_img(axes[0, 2], mean_of_samples[slice_index_to_show, 0][:, :].cpu().numpy(), f"(2) Mean of {num_samples} Samples")
plot_img(axes[0, 3], np.abs(abs_error_map[slice_index_to_show, 0][:, :].cpu().numpy()), "(5) Abs Error (Mean Sample vs SWA Mean)", cmap='inferno', vlims=None)

# Row 2
for i in range(num_random_samples_to_show):
    plot_img(axes[1, i], random_samples_to_show[i][slice_index_to_show, 0].cpu().numpy(), f"(3) Random Sample #{random_indices[i]+1}")

plot_img(axes[1, 3], np.abs(std_map[slice_index_to_show, 0].cpu().numpy()), "(4) STD Map (Uncertainty)", cmap='inferno', vlims=None)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

logger.info("--- END DIAGNOSTIC ---")